In [166]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, root_mean_squared_error
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.compose import ColumnTransformer

In [167]:
url = "https://raw.githubusercontent.com/rhodes-byu/stat-486/refs/heads/main/data/OceanicFisheries/ocean_data.csv"
df = pd.read_csv(url)

In [168]:
df_numerics = df.drop(["Cloud_Typ", "Wea", "Visibility", "Cloud_Amt"], axis=1)

In [169]:
df_numerics

,T_degC,Salnty,O2ml_L,Depthm,Bottom_D,Wind_Spd,Dry_T,Wet_T
0,16.830,33.8510,5.560,65,1337.0,14.0,16.5,15.5
1,9.262,33.8481,2.729,140,1202.0,5.0,15.0,13.0
2,15.390,33.4260,5.990,30,3871.0,10.0,18.8,17.6
3,14.540,32.9470,5.840,42,4018.0,14.0,16.9,16.1
4,7.410,34.1810,1.000,300,4058.0,21.0,16.3,14.9
...,...,...,...,...,...,...,...,...
7101,18.210,33.5170,6.940,2,62.0,10.0,20.2,19.0
7102,13.040,33.2110,5.570,113,4482.0,18.0,18.2,15.2
7103,12.170,33.4670,5.060,125,4125.0,24.0,15.0,12.5
7104,19.920,33.2580,5.390,50,4005.0,0.0,20.6,20.1


In [170]:
X = df_numerics.drop("T_degC", axis=1)
y = df_numerics["T_degC"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=307)

In [171]:
pipe = Pipeline([
    ('impute', SimpleImputer(strategy="mean")),
    ('polynomials', PolynomialFeatures(degree=2, include_bias=False)),
    ('standardize', StandardScaler()),
    ('model', LinearRegression())
])

In [172]:
pipe.fit(X_train, y_train)
yhat = pipe.predict(X_test)
yhat_training = pipe.predict(X_train)

In [173]:
# 2c
mean_squared_error(y_train, yhat_training)

1.5350956362420858

In [174]:
# 2d
mean_squared_error(y_test, yhat)

1.754330861094835

In [175]:
# 2e
np.var(y_test)

np.float64(14.8162872136925)

In [176]:
coefs = pipe.named_steps['model'].coef_
feature_names = pipe.named_steps['polynomials'].get_feature_names_out(input_features=X_test.columns)
coef_series = pd.Series(coefs, index=feature_names)

In [177]:
# 3a-c
coef_series.sort_values(key=abs, ascending=False)

Salnty Depthm        147.569766
Depthm              -146.550613
O2ml_L               -37.244657
Salnty Dry_T          35.153778
Dry_T                -34.383804
Salnty O2ml_L         32.217666
Wet_T                -11.009095
Salnty Wet_T           8.948392
Salnty^2              -4.503595
Bottom_D              -3.967908
Depthm Dry_T          -3.526225
Salnty Bottom_D        3.238598
O2ml_L Wet_T           2.673665
O2ml_L Dry_T           2.590504
Wind_Spd              -2.281111
Salnty Wind_Spd        1.990912
Depthm Wet_T           1.931986
Bottom_D Dry_T         1.922931
Wet_T^2                1.572198
Bottom_D Wet_T        -1.140666
O2ml_L^2               1.094337
Dry_T Wet_T           -0.857723
O2ml_L Depthm         -0.672363
O2ml_L Bottom_D        0.459726
Depthm Bottom_D       -0.345646
Dry_T^2               -0.281215
Wind_Spd Wet_T         0.264296
Depthm^2               0.140863
Wind_Spd Dry_T        -0.124319
Bottom_D Wind_Spd      0.124131
O2ml_L Wind_Spd        0.120860
Depthm W

In [178]:
# 3d
pipe.named_steps['model'].intercept_

np.float64(11.607865265528682)

In [179]:
pipe = Pipeline([
    ('impute', SimpleImputer(strategy="mean")),
    ('polynomials', PolynomialFeatures(degree=2, include_bias=False)),
    ('standardize', StandardScaler()),
    ('model', KNeighborsRegressor())
])

params = {
    'impute__strategy':('mean', 'median'),
    'polynomials__degree':(1,2,3),
    'model__n_neighbors':[5*x for x in range(1,21)],
    'model__weights':('uniform', 'distance')
}

gs = GridSearchCV(pipe, param_grid = params, scoring='neg_mean_squared_error')

In [180]:
gs.fit(X_train, y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...Regressor())])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'impute__strategy': ('mean', ...), 'model__n_neighbors': [5, 10, ...], 'model__weights': ('uniform', ...), 'polynomials__degree': (1, ...)}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'neg_mean_squared_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",None
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the co

In [181]:
# 4c1
print(gs.best_params_)

{'impute__strategy': 'median', 'model__n_neighbors': 10, 'model__weights': 'distance', 'polynomials__degree': 3}


In [182]:
# 4c2
print(gs.best_score_)

-1.2262958160647215


In [183]:
# 4ef
y_hat_gs = gs.predict(X_test)
print(mean_squared_error(y_test, y_hat_gs))


1.2398437245052845


In [184]:
rs = RandomizedSearchCV(pipe, param_distributions=params, scoring='neg_mean_squared_error')

In [185]:
rs.fit(X_train, y_train)

,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...Regressor())])
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'impute__strategy': ('mean', ...), 'model__n_neighbors': [5, 10, ...], 'model__weights': ('uniform', ...), 'polynomials__degree': (1, ...)}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",10
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'neg_mean_squared_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strateg

In [186]:
# 5c1
print(rs.best_params_)

{'polynomials__degree': 3, 'model__weights': 'uniform', 'model__n_neighbors': 10, 'impute__strategy': 'median'}


In [187]:
# 5c2
print(rs.best_score_)

-1.2645551497303984


In [188]:
# 5f
y_hat_rs = rs.predict(X_test)
print(mean_squared_error(y_test, y_hat_rs))

1.2696330679628585


In [189]:
pipe = Pipeline([
    ('impute', SimpleImputer(strategy="mean")),
    ('polynomials', PolynomialFeatures(degree=2, include_bias=False)),
    ('standardize', StandardScaler()),
    ('model', KNeighborsRegressor(n_neighbors=20, weights='distance'))
])

pipe.fit(X_train, y_train)
y_hat_train_old = pipe.predict(X_test)
print("Test MSE: ", mean_squared_error(y_test, y_hat_train_old))

Test MSE:  1.444862467860937


In [190]:
X = df.drop("T_degC", axis=1)
y = df["T_degC"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=307)

In [191]:
numeric_features = ['Salnty', 'O2ml_L', "Depthm", "Bottom_D", "Wind_Spd",
                    'Dry_T', "Wet_T"]
numeric_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('polys', PolynomialFeatures(degree=2, include_bias=False)),
    ('standardize', StandardScaler())
])

categorical_features = ["Wea", "Cloud_Typ", "Cloud_Amt", "Visibility"]
categorical_transformer = Pipeline([
    ('impute', SimpleImputer(strategy="most_frequent")),
    ('dummy', OneHotEncoder(sparse_output = False, handle_unknown='ignore')),
    ('dim_red', SelectPercentile(f_regression, percentile=50)) 
])

preprocessor = ColumnTransformer([
    ("num", numeric_pipe, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

In [192]:
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ("model", KNeighborsRegressor(n_neighbors=20, weights='distance'))
])

In [193]:
pipe.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers co

In [194]:
y_hat_train = pipe.predict(X_train)
y_hat_test = pipe.predict(X_test)
mse_train = mean_squared_error(y_train, y_hat_train)
mes_test = mean_squared_error(y_test, y_hat_test)
print("Training MSE: ", mse_train)
print("Test MSE: ", mes_test)

Training MSE:  7.716784388216377e-13
Test MSE:  1.462377671136538


In [195]:
import joblib

joblib.dump(pipe, "lab-03-final-model.joblib")

['lab-03-final-model.joblib']